In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

img1 = plt.imread('box_left_2k.jpg')
img2 = plt.imread('box_right_2k.jpg')

pts1 = np.array([[1148, 351], [1441, 481], [1427, 662], [1172, 525], [1524,80], [1795,134], [1744,307],
        [1564, 320], [1579, 306], [1584, 288]])

pts2 = np.array([[747, 397], [944,522], [1010,704], [822,558], [1212,116], [1440,170], [1433,350],
        [1131,360], [1152,342], [1166,326]])

K1 = np.array([[1062.04, 0, 1098.52], [0, 1061.74, 599.665], [0, 0, 1]])
K2 = np.array([[1060.59, 0, 1127.03], [0, 1060.06, 645.711], [0, 0, 1]])

# TODO 1 (15 points): Normalize image points.
#        There is no need to use for loops in this task. If you use for loops in this task, you can get at most 10 point.
# TODO 2 (25 points): Compute the fundamental matrix F.
# TODO 3 (15 points): Draw epipolar lines in image1.
# TODO 4 (10 points): Draw the epipole and epipolar lines in image1.
# TODO 5 (5 points): Compute the 4 possible camera matrices.
# TODO 6 (25 points): Triangulation.
#        There is no need to use for loops when computing the reconstruction errors of pts1 (or pts2).
#        If you do, you can get at most 20 point.
# TODO 7 (5 points): Visualize the 3D reconstruction.